In [ ]:
import os, sys
sys.path.append(os.path.dirname(os.path.dirname(os.getcwd())) + '/src/InitialisingEarth')
from TectonicEarth import Earth
earth = Earth()


print(os.path.dirname(os.getcwd()))
print(os.path.dirname(os.path.dirname(os.getcwd())) + '/src')

#sys.path.append(os.path.dirname())

In [ ]:
import os
import h5py
import stripy
import meshplex
import numpy as np
import pyvista as pv
from pathlib import Path
from scipy import interpolate
from PlateBoundaries import *
from TectonicEarth import Earth
from gospl.model import Model as sim
from gospl._fortran import definegtin
from GosplManager import GosplManager
from EarthsAssistant import EarthAssist

In [ ]:
earth = Earth(startTime=20, endTime=0, deltaTime=1)
earth.runTectonicSimulation()
earth.saveDataAsNPZ()

In [ ]:
earth = Earth(startTime=20, endTime=0, deltaTime=1)
earth.loadDataFromNPZ(1)

In [ ]:
earthMesh = earth.getEarthMesh()

plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh)
plotter.show()

In [ ]:
earth.animate(lookAtLonLat=[90, 20])

In [ ]:
gosplMan = GosplManager(earth, subdivisions=6)
gosplMesh = gosplMan.createGosplDataMesh(120, runNumber=76)

plotter = pv.PlotterITK()
plotter.add_mesh(gosplMesh, scalars='heights')
plotter.show()

In [ ]:
earth.animate(lookAtLonLat=[130, 40])

In [ ]:
gosplMan = GosplManager(earth, subdivisions=6)
gosplMan.makeDirectories()
gosplMan.createDomainNPdataFiles()
gosplMan.createTectonicDispNPZfiles()
gosplMan.runGosplSimulation()

In [ ]:
gosplMan.animateGosplOutput(lookAtLonLat=[130, 40])

In [ ]:
earth.animate(lookAtLonLat=[0, 0])

After running various Gospl runs, I found weird unrealeastic topological changes happening whenever I ran the Gospl simulation for too long (Eg From 40 Mya). This weird behaviour does not appear on shorter runs (Eg 5 Mya). A potential cause of this problem is because I only have one single backwards model running, whilst the Gospl BFModel notebook tutorial seems to use multiple backwards runs.

If adding multiple backwards gospl runs doesn't solve this issue, than the next solution would be to run many short but complete gospl simulations and simply stitch them together.

# Comparing Backward YML Files

In the BFModel tutorial, the file *backward15Ma.yml* seems to run first, followed by *backward10Ma.yml*. Apart from the domain and time section of the YML file, they seem to be similar:

##### backward15Ma.ym
``` yml
domain:
    npdata: 'input8/elev0Ma'
    flowdir: 5
    fast: True
    backward: True
    interp: 1
    overlap: 2

time:
    start: -15000000.
    end: -10000000.
    tout: 1000000.
    dt: 1000000.
    tec: 1000000.
```

##### backward10Ma.yml

``` yml
domain:
    npdata: 'input8/elev5Ma'
    flowdir: 5
    fast: True
    backward: True
    interp: 1
    overlap: 2

time:
    start: -10000000.
    end: 0.
    tout: 1000000.
    dt: 1000000.
    tec: 1000000.
```

I need to create a new elevation NPZ file every 5 Mya

In [ ]:
earth = Earth(
            startTime = 10,
            endTime = 0,
            deltaTime = 1,
            baseUplift = 2000,
            distTransRange = 1000000, 
            numToAverageOver = 10,
            earthRadius = 6378137.,
            useKilometres = False,
)

earth.runTectonicSimulation()

In [ ]:
earth.animate(lookAtLonLat=[80, 20], cameraZoom=1.4)

In [ ]:
gosplMan = GosplManager(earth, subdivisions=6)
gosplMan.createAllFilesAndRunSimulation()

In [ ]:
gosplMan.animateGosplOutput(lookAtLonLat=[80, 20])

In [ ]:
earthMesh = earth.getEarthMesh(iteration=20)

plotter = pv.PlotterITK()
plotter.add_mesh(earthMesh)
plotter.show()

In [ ]:
gosplFilenamePattern = gosplMan.thisRunDirectory + '/GosplOutputFiles/h5/gospl.{}.p0.h5'
gosplMesh = gosplMan.createGosplDataMesh(20)

plotter = pv.PlotterITK()
plotter.add_mesh(gosplMesh, scalars='heights')
plotter.show()